Election de 2022 premier tour

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import missingno as msno
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

import os
os.listdir()

['migrations',
 'election2017.ipynb',
 'pop-2007.ipynb',
 '.DS_Store',
 'populations',
 'population_2007_2023.xlsx',
 'alembic.ini',
 'enums',
 'utils',
 'models',
 'election2012.ipynb',
 'exports',
 'labo1.ipynb',
 '.gitignore',
 'scripts',
 'db',
 'chomages',
 'labo2.ipynb',
 '.git',
 'main.py',
 'election2022.ipynb',
 'elections']

#### Importer la database

In [20]:
from db import SessionLocal
session = SessionLocal()

Import des datasets.

In [21]:
tour = 2
df = pd.read_excel(f'./elections/presidentielles-2017-{tour}.xlsx')

Visualise toutes les colones

In [22]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(1000)

,Code du département,Libellé du département,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,% Blancs/Ins,% Blancs/Vot,Nuls,% Nuls/Ins,% Nuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot,Sexe,Nom,Prénom,Voix,% Voix/Ins,% Voix/Exp,Sexe.1,Nom.1,Prénom.1,Voix.1,% Voix/Ins.1,% Voix/Exp.1
0,1,Ain,415952,93135,22.39,322817,77.61,28467,6.84,8.82,9122,2.19,2.83,285228,68.57,88.36,M,MACRON,Emmanuel,173832,41.79,60.94,F,LE PEN,Marine,111396,26.78,39.06
1,2,Aisne,375791,90745,24.15,285046,75.85,22838,6.08,8.01,9067,2.41,3.18,253141,67.36,88.81,M,MACRON,Emmanuel,119202,31.72,47.09,F,LE PEN,Marine,133939,35.64,52.91
2,3,Allier,253481,59297,23.39,194184,76.61,18784,7.41,9.67,8634,3.41,4.45,166766,65.79,85.88,M,MACRON,Emmanuel,106568,42.04,63.90,F,LE PEN,Marine,60198,23.75,36.10
3,4,Alpes-de-Haute-Provence,126458,29254,23.13,97204,76.87,9605,7.60,9.88,3790,3.00,3.90,83809,66.27,86.22,M,MACRON,Emmanuel,48993,38.74,58.46,F,LE PEN,Marine,34816,27.53,41.54
4,5,Hautes-Alpes,109889,24892,22.65,84997,77.35,8669,7.89,10.20,2700,2.46,3.18,73628,67.00,86.62,M,MACRON,Emmanuel,47211,42.96,64.12,F,LE PEN,Marine,26417,24.04,35.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,ZN,Nouvelle-Calédonie,189368,89370,47.19,99998,52.81,6535,3.45,6.54,2344,1.24,2.34,91119,48.12,91.12,M,MACRON,Emmanuel,47902,25.30,52.57,F,LE PEN,Marine,43217,22.82,47.43
102,ZP,Polynésie française,203973,108330,53.11,95643,46.89,3273,1.60,3.42,2673,1.31,2.79,89697,43.97,93.78,M,MACRON,Emmanuel,52378,25.68,58.39,F,LE PEN,Marine,37319,18.30,41.61
103,ZS,Saint-Pierre-et-Miquelon,4963,2184,44.01,2779,55.99,306,6.17,11.01,155,3.12,5.58,2318,46.71,83.41,M,MACRON,Emmanuel,1467,29.56,63.29,F,LE PEN,Marine,851,17.15,36.71
104,ZW,Wallis et Futuna,8464,2369,27.99,6095,72.01,76,0.90,1.25,61,0.72,1.00,5958,70.39,97.75,M,MACRON,Emmanuel,4715,55.71,79.14,F,LE PEN,Marine,1243,14.69,20.86


Supprimer la colonne Etat saisie d’un DataFrame pandas.

In [23]:
df = df.drop(columns=["Etat saisie"], errors="ignore")

In [24]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(1000)

,Code du département,Libellé du département,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,% Blancs/Ins,% Blancs/Vot,Nuls,% Nuls/Ins,% Nuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot,Sexe,Nom,Prénom,Voix,% Voix/Ins,% Voix/Exp,Sexe.1,Nom.1,Prénom.1,Voix.1,% Voix/Ins.1,% Voix/Exp.1
0,1,Ain,415952,93135,22.39,322817,77.61,28467,6.84,8.82,9122,2.19,2.83,285228,68.57,88.36,M,MACRON,Emmanuel,173832,41.79,60.94,F,LE PEN,Marine,111396,26.78,39.06
1,2,Aisne,375791,90745,24.15,285046,75.85,22838,6.08,8.01,9067,2.41,3.18,253141,67.36,88.81,M,MACRON,Emmanuel,119202,31.72,47.09,F,LE PEN,Marine,133939,35.64,52.91
2,3,Allier,253481,59297,23.39,194184,76.61,18784,7.41,9.67,8634,3.41,4.45,166766,65.79,85.88,M,MACRON,Emmanuel,106568,42.04,63.90,F,LE PEN,Marine,60198,23.75,36.10
3,4,Alpes-de-Haute-Provence,126458,29254,23.13,97204,76.87,9605,7.60,9.88,3790,3.00,3.90,83809,66.27,86.22,M,MACRON,Emmanuel,48993,38.74,58.46,F,LE PEN,Marine,34816,27.53,41.54
4,5,Hautes-Alpes,109889,24892,22.65,84997,77.35,8669,7.89,10.20,2700,2.46,3.18,73628,67.00,86.62,M,MACRON,Emmanuel,47211,42.96,64.12,F,LE PEN,Marine,26417,24.04,35.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,ZN,Nouvelle-Calédonie,189368,89370,47.19,99998,52.81,6535,3.45,6.54,2344,1.24,2.34,91119,48.12,91.12,M,MACRON,Emmanuel,47902,25.30,52.57,F,LE PEN,Marine,43217,22.82,47.43
102,ZP,Polynésie française,203973,108330,53.11,95643,46.89,3273,1.60,3.42,2673,1.31,2.79,89697,43.97,93.78,M,MACRON,Emmanuel,52378,25.68,58.39,F,LE PEN,Marine,37319,18.30,41.61
103,ZS,Saint-Pierre-et-Miquelon,4963,2184,44.01,2779,55.99,306,6.17,11.01,155,3.12,5.58,2318,46.71,83.41,M,MACRON,Emmanuel,1467,29.56,63.29,F,LE PEN,Marine,851,17.15,36.71
104,ZW,Wallis et Futuna,8464,2369,27.99,6095,72.01,76,0.90,1.25,61,0.72,1.00,5958,70.39,97.75,M,MACRON,Emmanuel,4715,55.71,79.14,F,LE PEN,Marine,1243,14.69,20.86


In [25]:
df.shape

(106, 28)

Recuperer les information généraux sur les élections départemental

In [26]:
df_infos_general = df.iloc[:, :16]
df_infos_general.head()

,Code du département,Libellé du département,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,% Blancs/Ins,% Blancs/Vot,Nuls,% Nuls/Ins,% Nuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot
0,1,Ain,415952,93135,22.39,322817,77.61,28467,6.84,8.82,9122,2.19,2.83,285228,68.57,88.36
1,2,Aisne,375791,90745,24.15,285046,75.85,22838,6.08,8.01,9067,2.41,3.18,253141,67.36,88.81
2,3,Allier,253481,59297,23.39,194184,76.61,18784,7.41,9.67,8634,3.41,4.45,166766,65.79,85.88
3,4,Alpes-de-Haute-Provence,126458,29254,23.13,97204,76.87,9605,7.60,9.88,3790,3.00,3.90,83809,66.27,86.22
4,5,Hautes-Alpes,109889,24892,22.65,84997,77.35,8669,7.89,10.20,2700,2.46,3.18,73628,67.00,86.62


Renommer les colonnes des information généraux sur les élections départemental

In [27]:
df_infos_general.columns = [
    "code_dept", "nom_dept", "nb_inscrits", "nb_abstentions",
    "pct_abstentions", "nb_votants", "pct_votants", "nb_blancs",
    "pct_blancs_inscrits", "pct_blancs_votants", "nb_nuls",
    "pct_nuls_inscrits", "pct_nuls_votants", "nb_exprimes",
    "pct_exprimes_inscrits", "pct_exprimes_votants"
]
df_infos_general.head()

,code_dept,nom_dept,nb_inscrits,nb_abstentions,pct_abstentions,nb_votants,pct_votants,nb_blancs,pct_blancs_inscrits,pct_blancs_votants,nb_nuls,pct_nuls_inscrits,pct_nuls_votants,nb_exprimes,pct_exprimes_inscrits,pct_exprimes_votants
0,1,Ain,415952,93135,22.39,322817,77.61,28467,6.84,8.82,9122,2.19,2.83,285228,68.57,88.36
1,2,Aisne,375791,90745,24.15,285046,75.85,22838,6.08,8.01,9067,2.41,3.18,253141,67.36,88.81
2,3,Allier,253481,59297,23.39,194184,76.61,18784,7.41,9.67,8634,3.41,4.45,166766,65.79,85.88
3,4,Alpes-de-Haute-Provence,126458,29254,23.13,97204,76.87,9605,7.60,9.88,3790,3.00,3.90,83809,66.27,86.22
4,5,Hautes-Alpes,109889,24892,22.65,84997,77.35,8669,7.89,10.20,2700,2.46,3.18,73628,67.00,86.62


In [28]:
df_infos_general.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   code_dept              106 non-null    object 
 1   nom_dept               106 non-null    object 
 2   nb_inscrits            106 non-null    int64  
 3   nb_abstentions         106 non-null    int64  
 4   pct_abstentions        106 non-null    float64
 5   nb_votants             106 non-null    int64  
 6   pct_votants            106 non-null    float64
 7   nb_blancs              106 non-null    int64  
 8   pct_blancs_inscrits    106 non-null    float64
 9   pct_blancs_votants     106 non-null    float64
 10  nb_nuls                106 non-null    int64  
 11  pct_nuls_inscrits      106 non-null    float64
 12  pct_nuls_votants       106 non-null    float64
 13  nb_exprimes            106 non-null    int64  
 14  pct_exprimes_inscrits  106 non-null    float64
 15  pct_ex

Suppression des données redondantes, risque de multicolinéarité en ML

In [29]:
colonnes_base = [
    "code_dept", "nom_dept",
    "nb_inscrits", "nb_abstentions", "nb_votants", 
    "nb_blancs", "nb_nuls"
]

df_infos_general_base = df_infos_general[colonnes_base]
df_infos_general_base.head()

,code_dept,nom_dept,nb_inscrits,nb_abstentions,nb_votants,nb_blancs,nb_nuls
0,1,Ain,415952,93135,322817,28467,9122
1,2,Aisne,375791,90745,285046,22838,9067
2,3,Allier,253481,59297,194184,18784,8634
3,4,Alpes-de-Haute-Provence,126458,29254,97204,9605,3790
4,5,Hautes-Alpes,109889,24892,84997,8669,2700


In [30]:
df_infos_general_base.shape

(106, 7)

Si les colonnes nb_blancs et nb_nuls existent, les fusionées.

In [31]:

if {"nb_blancs", "nb_nuls"}.issubset(df_infos_general_base.columns):
    df_infos_general_base["nb_blancs_nuls"] = df_infos_general_base["nb_blancs"].fillna(0) + df_infos_general_base["nb_nuls"].fillna(0)
    df_infos_general_base = df_infos_general_base.drop(columns=["nb_blancs", "nb_nuls"])

df_infos_general_base.head()

/tmp/ipykernel_88498/3627291289.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_infos_general_base["nb_blancs_nuls"] = df_infos_general_base["nb_blancs"].fillna(0) + df_infos_general_base["nb_nuls"].fillna(0)


,code_dept,nom_dept,nb_inscrits,nb_abstentions,nb_votants,nb_blancs_nuls
0,1,Ain,415952,93135,322817,37589
1,2,Aisne,375791,90745,285046,31905
2,3,Allier,253481,59297,194184,27418
3,4,Alpes-de-Haute-Provence,126458,29254,97204,13395
4,5,Hautes-Alpes,109889,24892,84997,11369


In [32]:
df_infos_general_base.shape

(106, 6)

In [33]:
df_departement = df_infos_general_base.iloc[:, :2]
df_departement.head()

,code_dept,nom_dept
0,1,Ain
1,2,Aisne
2,3,Allier
3,4,Alpes-de-Haute-Provence
4,5,Hautes-Alpes


In [34]:
df_stat_elections = df_infos_general_base.drop(columns=["nom_dept"])
df_stat_elections.head()

,code_dept,nb_inscrits,nb_abstentions,nb_votants,nb_blancs_nuls
0,1,415952,93135,322817,37589
1,2,375791,90745,285046,31905
2,3,253481,59297,194184,27418
3,4,126458,29254,97204,13395
4,5,109889,24892,84997,11369


Tratification pour consever les proportions

In [35]:
import pandas as pd

# Colonnes globales que l'on veut garder
cols_globales = ['Code du département', 'Libellé du département']

# Colonnes répétitives candidats
df_candidates = df.iloc[:, 16:]  # après % Exp/Vot
cols_per_candidate = 6
num_candidates = df_candidates.shape[1] // cols_per_candidate

dfs_list = []

for i in range(num_candidates):
    start_col = i * cols_per_candidate
    end_col = start_col + cols_per_candidate
    df_cand = df_candidates.iloc[:, start_col:end_col].copy()
    df_cand.columns = ['Sexe', 'Nom', 'Prenom', 'Voix', '% Voix/Ins', '% Voix/Exp']
    
    # Ajouter les informations de département
    df_cand = pd.concat([df[cols_globales], df_cand], axis=1)
    
    dfs_list.append(df_cand)

# Combiner tous les candidats
df_simple = pd.concat(dfs_list, ignore_index=True)

# Colonnes à garder
cols_to_keep = ['Code du département', 'Libellé du département', 'Sexe', 'Nom', 'Prenom', 'Voix']

# Nouveau dataframe simplifié
df_candidat_resultat = df_simple[cols_to_keep]

# Afficher toutes les lignes
pd.set_option('display.max_rows', 10)  # ou un nombre supérieur à 100
pd.set_option('display.max_columns', None)  # pour voir toutes les colonnes
pd.set_option('display.width', None)        # pour éviter la coupure à la largeur de l'écran


# Affichage des 5 premières lignes
df_candidat_resultat.head(1000)


,Code du département,Libellé du département,Sexe,Nom,Prenom,Voix
0,1,Ain,M,MACRON,Emmanuel,173832
1,2,Aisne,M,MACRON,Emmanuel,119202
2,3,Allier,M,MACRON,Emmanuel,106568
3,4,Alpes-de-Haute-Provence,M,MACRON,Emmanuel,48993
4,5,Hautes-Alpes,M,MACRON,Emmanuel,47211
...,...,...,...,...,...,...
207,ZN,Nouvelle-Calédonie,F,LE PEN,Marine,43217
208,ZP,Polynésie française,F,LE PEN,Marine,37319
209,ZS,Saint-Pierre-et-Miquelon,F,LE PEN,Marine,851
210,ZW,Wallis et Futuna,F,LE PEN,Marine,1243


In [36]:
from datetime import date

from utils.election_importer import ElectionImporter
from enums.type_election import TypeElection

# =====================================================
# Instanciation
# =====================================================
importer = ElectionImporter()

# =====================================================
# Élection
# =====================================================
if tour == 1:
    election = importer.get_or_create_election(
        election_date=date(2017, 4, 23),  # 1er tour présidentielle 2017
        type_election=TypeElection.PRESIDENTIELLE,
        tour=tour
    )

elif tour == 2:
    election = importer.get_or_create_election(
        election_date=date(2017, 5, 7),  # 2nd tour présidentielle 2017
        type_election=TypeElection.PRESIDENTIELLE,
        tour=tour
    )

# =====================================================
# Départements
# =====================================================
#df_departement = edf.df_departement
if df_departement is not None and not df_departement.empty:
    importer.import_departements(df_departement)

# =====================================================
# Statistiques par département
# =====================================================
#df_stat_elections = edf.df_stat_elections
if df_stat_elections is not None and not df_stat_elections.empty:
    importer.import_stats(df_stat_elections, election.id)

# =====================================================
# Candidats & résultats
# =====================================================
#df_candidat_resultat = edf.df_candidat_resultat
if df_candidat_resultat is not None and not df_candidat_resultat.empty:
    importer.import_candidats_resultats(df_candidat_resultat, election.id)

print("✅ Import élection 2017 – Tour 1 terminé avec succès")


2026-01-04 07:42:22,647 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-04 07:42:22,649 INFO sqlalchemy.engine.Engine SELECT elections.id AS elections_id, elections.date AS elections_date, elections.type_election AS elections_type_election, elections.tour AS elections_tour 
FROM elections 
WHERE elections.date = %(date_1)s AND elections.type_election = %(type_election_1)s AND elections.tour = %(tour_1)s 
 LIMIT %(param_1)s
2026-01-04 07:42:22,650 INFO sqlalchemy.engine.Engine [cached since 13.09s ago] {'date_1': datetime.date(2017, 5, 7), 'type_election_1': 'PRESIDENTIELLE', 'tour_1': 2, 'param_1': 1}
2026-01-04 07:42:22,664 INFO sqlalchemy.engine.Engine INSERT INTO elections (date, type_election, tour) VALUES (%(date)s, %(type_election)s, %(tour)s) RETURNING elections.id
2026-01-04 07:42:22,666 INFO sqlalchemy.engine.Engine [cached since 13.1s ago] {'date': datetime.date(2017, 5, 7), 'type_election': 'PRESIDENTIELLE', 'tour': 2}
2026-01-04 07:42:22,670 INFO sqlalchemy.engine.En